In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate
import numpy as np
from peft import get_peft_model, TaskType, LoraConfig, PeftModel

In [ ]:
MODEL="distilbert/distilbert-base-uncased"
DATASET="dair-ai/emotion"
id2label = {0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"}
label2id = {v:k for k,v in id2label.items()}
METRIC="accuracy"
TASK='text-classification'
NUM_LABELS=len(id2label)

In [3]:
data = load_dataset(DATASET)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenized_data = data.map(lambda d: tokenizer(d["text"], truncation=True), batched=True, num_proc=8)
tokenized_data['train']

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 16000
})

In [5]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load(METRIC)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=NUM_LABELS, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
task_evaluator = evaluate.evaluator(TASK)
task_evaluator.compute(model, data['test'].select(range(1000)), METRIC, tokenizer=tokenizer, label_mapping=label2id)

`data` is a preloaded Dataset! Ignoring `subset` and `split`.
Device set to use cuda:0


{'accuracy': 0.117,
 'total_time_in_seconds': 2.482303395998315,
 'samples_per_second': 402.85164239475534,
 'latency_in_seconds': 0.0024823033959983152}

In [8]:
training_args = TrainingArguments(
    output_dir="temp",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    bf16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.258100,0.219850,0.920500
2,0.147200,0.180332,0.926500


TrainOutput(global_step=2000, training_loss=0.329415714263916, metrics={'train_runtime': 46.2772, 'train_samples_per_second': 691.485, 'train_steps_per_second': 43.218, 'total_flos': 389287358125632.0, 'train_loss': 0.329415714263916, 'epoch': 2.0})

In [9]:
task_evaluator.compute(model, data['test'].select(range(1000)), METRIC, tokenizer=tokenizer, label_mapping=label2id)

`data` is a preloaded Dataset! Ignoring `subset` and `split`.
Device set to use cuda:0


{'accuracy': 0.933,
 'total_time_in_seconds': 3.224295036001422,
 'samples_per_second': 310.1453151260439,
 'latency_in_seconds': 0.0032242950360014217}

In [56]:
peft_config = LoraConfig(r=8, task_type=TaskType.SEQ_CLS, target_modules=["q_lin", "k_lin","v_lin"])
model1 = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=NUM_LABELS, id2label=id2label, label2id=label2id)
model1 = get_peft_model(model1, peft_config)
model1.print_trainable_parameters()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 816,390 || all params: 67,774,476 || trainable%: 1.2046


In [57]:
training_args = TrainingArguments(
    output_dir="temp",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    bf16=True,
)

trainer = Trainer(
    model=model1,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.048000,0.847556,0.701500
2,0.710400,0.624531,0.774500
3,0.601700,0.538718,0.809000
4,0.532800,0.486875,0.825000
5,0.497900,0.447924,0.835500
6,0.458800,0.422055,0.844000
7,0.432300,0.403087,0.854500
8,0.416100,0.389850,0.861500
9,0.419800,0.382297,0.859500
10,0.417200,0.380064,0.861500


TrainOutput(global_step=10000, training_loss=0.5862501937866211, metrics={'train_runtime': 178.6632, 'train_samples_per_second': 895.54, 'train_steps_per_second': 55.971, 'total_flos': 1982445802331904.0, 'train_loss': 0.5862501937866211, 'epoch': 10.0})

In [58]:
model1.save_pretrained("lora_weights")

In [59]:
model2 = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=NUM_LABELS, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [60]:
model2 = PeftModel.from_pretrained(model2, "./lora_weights")
model2 = model2.merge_and_unload()

In [61]:
task_evaluator.compute(model2, data['test'].select(range(1000)), METRIC, tokenizer=tokenizer, label_mapping=label2id)

`data` is a preloaded Dataset! Ignoring `subset` and `split`.
Device set to use cuda:0


{'accuracy': 0.865,
 'total_time_in_seconds': 2.33021377499972,
 'samples_per_second': 429.1451757468562,
 'latency_in_seconds': 0.00233021377499972}